# Grenzen des Maschinellen Lernens

Das Konzept p-Hacking beschreibt, wie man absichtlich so lange die Auswertung der eigenen Daten verändert, bis auf einmal das Modell eine sehr hohe Güte erreicht, ohne dass es dafür eine inhaltliche Erklärung gibt.
Dieses Konzept stammt aus der Statistik, wo ein zuvor festgeleget p-Wert unterschritten werden soll, was angeblich eine besonders hohe Güte des Modells belegen soll.
Beim Maschinellen Lernen werden andere Werte, wie z. B. die Genauigkeit, zum Messen verwendet - so besagt eine hohe Genauigkeit, dass das Modell eine hohe Güte hat.
Deswegen haben wir es strenggenommen im Folgenden mit Genauigkeits-Hacking zu tun.
Aber ist es wirklich gut, sich alleine auf eine Metrik (p-Wert, Genauigkeit, ...) zu verlassen?

Um besonders gute Genauigkeitswerte zu erhalten, können z. B. Scheinkorrelationen oder andere nicht-lineare zufällige Zusammenhänge ausgenutzt werden.
Wenn genügend Daten gleichzeitig betrachtet werden, gibt es mit hoher Wahrscheinlichkeit ein Attribut, welches einen Zusammenhang mit der Zielvariable aufweist.
Deswegen sollte immer sehr kritisch betrachtet werden, aus welchen Daten welches Ergebnis angeblich vorhersagbar sein soll.

So Verhalten ist ein Problem für alle Bereiche:
In der Wissenschaft werden unter Umständen falsche Annahmen und Vorhersagemodelle verwendet.
In der Praxis bedeutet es, dass ggf. defekte Vorhersagemodelle in den Betrieb aufgenommen werden.
Sobald sich jemand auf die Vorhersagen des falschen Modells verlässt, kann dies zu ernsthaften Schäden an Menschen oder Umwelt führen.
Ebenso sind finanzielle Schäden nicht ausgeschlossen.

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

Es wird zunächst ein Datensatz mit zufälligen Attribute erzeugt.
Dies könnte z. B. die Zahlenrepräsentation von nominalskalierten Attributen sein.

In [ ]:
# Fixiere Zufallsgenerator
random.seed(0)

number_rows = 20

df = pd.DataFrame({
    "Outlook": [random.choice(["Sunny", "Overcast", "Rain"]) for _ in range(number_rows)],
    "Temperature": [random.choice(["Hot", "Mild", "Cool"]) for _ in range(number_rows)],
    "Humidity": [random.choice(["High", "Normal"]) for _ in range(number_rows)],
    "Wind": [random.choice(["Weak", "Strong"]) for _ in range(number_rows)],
    "Play Tennis?": [random.choice(["Yes", "No"]) for _ in range(number_rows)],
})

df

In [ ]:
df_cat = df.assign(**{
    col : df[col].astype('category').cat.codes for col in ["Humidity", "Wind", "Play Tennis?"]
})
df_cat = df_cat.assign(
    Outlook=df["Outlook"].astype(
        pd.CategoricalDtype(categories=["Rain", "Overcast", "Sunny"], ordered=True)).cat.codes,
    Temperature=df["Temperature"].astype(
        pd.CategoricalDtype(categories=["Cool", "Mild", "Hot"], ordered=True)).cat.codes    
)
df_cat

Nun lass uns betrachten, ob der Entscheidungsbaum einen zufälligen Zusammenhang findet, und das Kreuzvalidierungsergebnis dennoch gut aussieht.

In [ ]:
import sklearn.tree
dt = sklearn.tree.DecisionTreeClassifier()
eingabe = df_cat[list(set(df_cat.columns) - set(["Play Tennis?"]))]

ziel = df["Play Tennis?"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(eingabe, ziel, random_state=42)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

<span style="color:blue; font-weight:bold">Aufgabe 1</span>

Interpretieren Sie den Zahlenwert.
Liegt hier eine Scheinkorrelation vor?
Ist dieses Ergebnis plausibel?
Warum (nicht)?

*Antwort*: ...

## Feature Engineering falsch gemacht

Normalerweise hilft Feature Engineering dabei, zielführende Zahlenwerte zu generieren.
So ist es z. B. sinnvoll, statt einem Start- und einem Endzeitpunkt lieber gleich die Zeitspanne zu generieren, oder statt einer Länge und einer Breite gleich die Fläche zu berechnen.
Es gibt aber auch Fälle, in denen Projektverantwortliche beliebige Attribute addiert, multipliziert etc. haben, ohne dass es dafür eine inhaltliche Rechtfertigung gab.
Genau das werden wir jetzt auf die Spitze treiben.

Nun werden wir so lange neue Attribute erstellen, bis wir (mindestens) eines finden, mit dem der Entscheidungsbaum zu einem guten Ergebnis kommt.

In [ ]:
# Hier werden die Additionen zweier Attribute gespeichert
new_columns = {}

for column_A in list(set(df_cat.columns) - set(["Play Tennis?"])):
    for column_B in list(set(df_cat.columns) - set(["Play Tennis?"])):
        if column_A == column_B:
            continue
            
        addition = df_cat[column_A] + df_cat[column_B]
        addition.name = f"{column_A} + {column_B}"
        new_columns.update({addition.name : addition})

        multiplication = df_cat[column_A] * df_cat[column_B]
        multiplication.name = f"{column_A} * {column_B}"
        new_columns.update({multiplication.name : multiplication})


Diese kreierten Attribute fügen wir nun dem DataFrame hinzu.

In [ ]:
df_extended = df_cat.assign(**new_columns)

df_extended

Nun lass uns sehen, ob der Entscheidungsbaum mit diesen generierten Features besser arbeiten kann:

In [ ]:
ziel = df_extended["Play Tennis?"]
eingabe = df_extended[list(set(df_extended.columns) - set(["Play Tennis?"]))]

dt = sklearn.tree.DecisionTreeClassifier(random_state=1)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(eingabe, ziel, random_state=6)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

<span style="color:blue; font-weight:bold">Aufgabe 2</span>

Interpretieren Sie den Zahlenwert.
Liegt hier eine Scheinkorrelation vor?
Ist dieses Ergebnis plausibel?
Warum (nicht)?

*Antwort*: ...

<span style="color:blue; font-weight:bold">Aufgabe 3</span>

Stellen Sie sich vor, ein Dienstleister würde Ihnen das obige Ergebnis präsentieren.
Wären Sie mit dem Ergebnis zufrieden?
Warum (nicht)?

*Antwort*: ...

<span style="color:blue; font-weight:bold">Aufgabe 4</span>

Mit welchen Maßnahmen können Sie sicherstellen, dass ein Dienstleister Sie nicht mit Tricks wie P-Hacking hereinlegen kann?
Gehen Sie davon aus, dass Sie den Source Code nicht einsehen können.

*Antwort*: ...

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.